In [144]:
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV,KFold
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, RidgeCV, Lasso, ElasticNet
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

### Import Dataset

In [80]:
test=pd.read_csv('test_data_id.csv')
train=pd.read_csv('train_data_id.csv')
train.head()

,X1,X2,X3,X4,X12,X13,X14,X23,X24,X34,X123,X124,X234,X1234,X5
0,6.7,240,0.516,0.728,1608.0,3.4572,4.8776,123.840,174.720,0.375648,829.7280,1170.6240,90.15552,604.041984,8.9
1,7.3,263,0.482,0.655,1919.9,3.5186,4.7815,126.766,172.265,0.315710,925.3918,1257.5345,83.03173,606.131629,7.2
2,6.3,185,0.374,0.709,1165.5,2.3562,4.4667,69.190,131.165,0.265166,435.8970,826.3395,49.05571,309.050973,8.1
3,6.8,210,0.402,0.739,1428.0,2.7336,5.0252,84.420,155.190,0.297078,574.0560,1055.2920,62.38638,424.227384,8.7
4,5.8,180,0.425,0.872,1044.0,2.4650,5.0576,76.500,156.960,0.370600,443.7000,910.3680,66.70800,386.906400,11.8


In [81]:
#sns.pairplot(data=train, diag_kind='kde')

In [82]:
#sns.heatmap(train[['X2', 'X3', 'X4']].corr(),annot=True)
#plt.show()

In [126]:
X_train=train.iloc[:44,:14]
Y_train=train.iloc[:44,14]


### Variable Selection procedure 

In [127]:
rfe = RFECV(estimator=LinearRegression(),min_features_to_select=3)

In [128]:
Reg_var = rfe.fit(X_train,Y_train)

In [129]:
Reg_var.support_

array([False, False,  True,  True, False, False, False, False, False,
        True, False, False, False, False])

In [130]:
X_train=train[['X3','X4','X34']]
X_test=test[['X3','X4','X34']]

### Model Fitting

In [131]:
model=LinearRegression()

In [132]:
m=model.fit(X_train,(Y_train))

In [133]:
m.intercept_

-71.93798042586431

In [134]:
m.coef_

array([ 160.99554659,   91.82821341, -171.09647741])

In [135]:
m.score(X_train,(Y_train))

0.20846502213621343

### Predictions

In [136]:
y_pred=model.predict(X_test)
(y_pred)

array([11.96387519, 11.55789283, 13.34395294, 12.94719841, 13.74794741,
       11.73873058,  6.30319272, 12.5580987 , 14.2441941 , 11.64663281,
       11.38963408])

### Lasso, Ridge, ElasticNet Models

In [186]:
x=train[['X3','X4','X34']]
y=train.iloc[:,14]
x_test_data=test[['X3','X4','X34']]

In [187]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [188]:
model1=Lasso()
params={"alpha":[0.08,0.09,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,2,4,6,7,8,9,10,20,50,100 ]}
cval=KFold(n_splits=10)

In [189]:
model3=Ridge()
params={"alpha":[0.08,0.09,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,2,4,6,7,8,9,10,20,50,100 ]}
cval=KFold(n_splits=10)

In [190]:
model2=ElasticNet()
params={"alpha":[0.08,0.09,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,2,4,6,7,8,9,10,20,50,100 ]}
cval=KFold(n_splits=10)

In [191]:
gsearch=GridSearchCV(model1,params,cv=cval)
esearch=GridSearchCV(model2,params,cv=cval)
rsearch=GridSearchCV(model3,params,cv=cval)

In [192]:
results=gsearch.fit(x_train,y_train)
results.best_params_

{'alpha': 0.3}

In [193]:
results=esearch.fit(x_train,y_train)
results.best_params_

{'alpha': 0.5}

In [194]:
results=rsearch.fit(x_train,y_train)
results.best_params_

{'alpha': 20}

In [195]:
lmodel=Lasso(alpha=0.0001)
lmodel.fit(x_train,y_train)

emodel=ElasticNet(alpha=0.0001) #0.0000000000001
emodel.fit(x_train,y_train)

rmodel=Ridge(alpha=0.0001) #0.0000000000001
rmodel.fit(x_train,y_train)

C:\Users\user\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 111.75892123933562, tolerance: 0.0884129411764706
  model = cd_fast.enet_coordinate_descent(


Ridge(alpha=0.0001)

#### Lasso Model

In [196]:
lmodel.coef_

array([ 161.80636243,   97.59329005, -178.04745855])

In [197]:
lmodel.score(x_train,y_train)

0.23422944880539365

In [198]:
ly_pred=(lmodel.predict(x_test))
MSE=mean_squared_error(ly_pred,y_test)
MSE

37.2987995569082

In [199]:
(lmodel.predict(x_test_data))

array([12.39040027, 11.95625791, 13.62566875, 12.97517821, 14.09639711,
       12.32044569,  6.20446293, 12.64309745, 14.21624667, 11.80604321,
       11.72569296])

#### Ridge Model

In [200]:
rmodel.coef_

array([ 134.12560266,   80.10815106, -140.19310027])

In [201]:
rmodel.score(x_train,y_train)

0.23257982398680166

In [202]:
ry_pred=emodel.predict(x_test)
MSE=mean_squared_error(ry_pred,y_test)
MSE

36.978745286128586

In [207]:
rmodel.predict(x_test_data)

array([12.33838562, 11.91417509, 13.70484516, 13.00238961, 14.2198792 ,
       12.18095398,  6.5483167 , 12.66679267, 14.37821053, 11.82048909,
       11.70728859])

#### ElasticNet Model

In [208]:
emodel.coef_

array([38.65849254, 19.89310131, -9.99741727])

In [209]:
emodel.score(x_train,y_train)

0.21606643535533232

In [210]:
ey_pred=emodel.predict(x_test)
MSE=mean_squared_error(ey_pred,y_test)
MSE

36.978745286128586

In [211]:
emodel.predict(x_test_data)

array([12.15715879, 11.76946494, 13.96087514, 13.08292974, 14.62447428,
       11.70366277,  7.75321321, 12.73823351, 14.90907878, 11.86692117,
       11.64419938])